In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
MAX_VAL = 10
NUM_TRIALS = 10000

In [3]:
def create_population(num_people, max_val):
    return np.random.randint(low=0, high=max_val, size=num_people)

def get_random_projection(population_sample, output_dimension, random_proj_scale=1):
    random_projection = random_proj_scale*np.random.random((len(population_sample), output_dimension))
    return random_projection


def oracle(random_projection, population_sample):
    '''The oracle takes a population of integers and 
    projects them to some k dimension space. This is just a fake
    oracle
    '''
    return population_sample.T @ random_projection

def sum_oracle(population_sample):
    
    return sum(population_sample)

def laplace_mechanism(oracle_output, epsilon, sensitivity):

    scale = sensitivity/epsilon
    
    if isinstance(oracle_output, np.int64) or isinstance(oracle_output, np.float64):
        return oracle_output +  np.random.laplace(loc=0.0, scale=scale, size=None)
    
    z = np.zeros(len(oracle_output))
    for i in range(len(oracle_output)):
        z[i] = oracle_output[i] + np.random.laplace(loc=0.0, scale=scale, size=None)
        
    return z 


In [4]:
def get_error(y, y_hat):
    
    if isinstance(y, np.int64) or isinstance(y, np.float64):
        return abs(y - y_hat)

    return np.linalg.norm((y - y_hat), ord=np.inf)

def bad_event(y, y_hat, sensitivity, epsilon, output_dim, beta):
    '''1 if a bad event has occurred 0 otherwise
    '''
    error = get_error(y, y_hat) 
    theoretical_bound = np.log(output_dim/beta)*(sensitivity/epsilon)
#     print("Actual error: {:.2f} Theory Error: {:.2f}".format(error, theoretical_bound))
    return error >= theoretical_bound

In [8]:
# Experiment: Verifying empirically the tightness of alpha beta accuracy of laplace noise
# So there is a monte carlo error in this as well but for large number
# of trials theory says monte carlo estimate should be very close
# to real probability of error
for num_trials in ([100, 1000, 10000, 100000]):
    num_people = 1000
    max_val = MAX_VAL
    X = create_population(num_people, max_val)
    k = 5

    proj_scale = 1
    temp = get_random_projection(X, k, proj_scale)
    oracle_output = oracle(temp, X)

    epsilon = 0.01
    sensitivity = max_val
    beta = 0.05 # Classic H-Testing number
    count = 0
    for trial in range(num_trials):
        private_output = laplace_mechanism(oracle_output, epsilon, sensitivity)
        count += bad_event(oracle_output, 
                                     private_output, 
                                     sensitivity, 
                                     epsilon, 
                                     k, 
                                     beta)
    print("Num trials", num_trials)
    print("Monte carlo estimate of bad prob: {:.3f}\nExpected value: {:.3f}\n".format(count/num_trials, beta))

Num trials 100
Monte carlo estimate of bad prob: 0.040
Expected value: 0.050

Num trials 1000
Monte carlo estimate of bad prob: 0.045
Expected value: 0.050

Num trials 10000
Monte carlo estimate of bad prob: 0.048
Expected value: 0.050

Num trials 100000
Monte carlo estimate of bad prob: 0.049
Expected value: 0.050



In [10]:
# Experiment: Verifying empirically the tightness of alpha beta accuracy of laplace noise
# So there is a monte carlo error in this as well but for large number
# of trials theory says monte carlo estimate should be very close
# to real probability of error. k=1 for sum queries
for num_trials in ([10000, 100000]):
    num_people = 1000
    max_val = MAX_VAL
    X = create_population(num_people, max_val)
    k = 1

    oracle_output = sum_oracle(X)

    epsilon = 0.01
    sensitivity = max_val
    beta = 0.05 # Classic H-Testing number
    count = 0
    for trial in range(num_trials):
        private_output = laplace_mechanism(oracle_output, epsilon, sensitivity)
        count += bad_event(oracle_output, 
                                     private_output, 
                                     sensitivity, 
                                     epsilon, 
                                     k, 
                                     beta)
    print("Num trials", num_trials)
    print("Monte carlo estimate of bad prob: {:.3f}\nExpected value: {:.3f}\n".format(count/num_trials, beta))

Num trials 10000
Monte carlo estimate of bad prob: 0.052
Expected value: 0.050

Num trials 100000
Monte carlo estimate of bad prob: 0.051
Expected value: 0.050

